# Load translation voxel simulation data

In [1]:
%%time
%load_ext autoreload

from analysis_utils import *
from joblib import Parallel, delayed
import multiprocessing  
import numpy as np
import pandas as pd

#Growth rate = 3.0
#Computes the translation latencies from our model with simulations containing
#1 to 14 cognate ternary complexes in the voxel (14 to 42 not included because
#the probability of voxels with that number of cognates is ~0)
path = "/Users/Akshay/Documents/tRNAShuffle/data/"
ensmbl_latency_dict = dict()
rxndiff=dict()
scalingFactor = 200
NR_scaledrates = {'k1r':718*200/scalingFactor,'k2f':1475*200/scalingFactor,'k2r_nr':1120*200/scalingFactor,'k3_nr':6*200/scalingFactor,'k4':209*200/scalingFactor}


data = "191003_2210/"
for i in range(1,7):
    sim_i = computeTransportRxnTimes(path+data,100*(i-1),100*(i-1)+100,cogtRNANum=i, ribosomeNum = 7,scaling=scalingFactor,NR_scaling = NR_scaledrates)
    ensmbl_latency_dict[i] = (CellLatencies(sim_i))

data = "191105_0908/"
for i in range(7,14):
    sim_i = computeTransportRxnTimes(path+data, 100*(i-1-6),100*(i-1-6)+100,cogtRNANum=i, ribosomeNum = 7,scaling=scalingFactor,NR_scaling = NR_scaledrates)
    ensmbl_latency_dict[i] = (CellLatencies(sim_i))
    
#Experimentally obtained codon usage frequencies and tRNA abundances at growth rate = 2.5 dbl/hr
gr25_Codon = np.array([2.36, 1.26, 45.55, 34.17, 16.97, 57.86, 19.27, 33.74, 14.98, 22.31, 43.18, 7.67, 24.11, 24.87, 39.49, 11.81, 0.03, 0.63, 2.19, 9.31, 17.22, 55.01, 5.61, 29.21, 21.67, 0.52, 15.79, 43.86, 4.17, 2.61, 20.64, 26.7, 7.03, 0.19, 2.76, 3.81, 6.72, 16.52, 4.27, 2.73, 7.92, 23.25, 2.51, 1.98, 16.33, 11.68, 0.62, 0.67, 43.82, 20.59, 27.28, 7.01, 6.78, 14.21, 60.75, 0.82, 3.86, 4.09, 28.82, 5.18, 4.38, 1.09])
gr25_pCodon = gr25_Codon/np.sum(gr25_Codon)
gr25_ptRNA = [0.0602409638554217, 0.010542168674698798, 0.07379518072289158, 0.00602409638554217, 0.010542168674698798, 0.00602409638554217, 0.021084337349397596, 0.043674698795180725, 0.021084337349397596, 0.01204819277108434, 0.01807228915662651, 0.08433734939759038, 0.03162650602409639, 0.07228915662650603, 0.01204819277108434, 0.07078313253012049, 0.06325301204819278, 0.01656626506024097, 0.009036144578313254, 0.027108433734939763, 0.010542168674698798, 0.03012048192771085, 0.013554216867469882, 0.015060240963855425, 0.0075301204819277125, 0.010542168674698798, 0.0075301204819277125, 0.003012048192771085, 0.021084337349397596, 0.004518072289156627, 0.01656626506024097, 0.01204819277108434, 0.0015060240963855425, 0.009036144578313254, 0.01656626506024097, 0.019578313253012052, 0.015060240963855425, 0.027108434, 0.05873493975903616, 0.019578313]
gr25_ptRNA = gr25_ptRNA/np.sum(gr25_ptRNA)

n_cores = 16

Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
CPU times: user 45.8 s, sys: 640 ms, total: 46.5 s
Wall time: 47.3 s


## Uniform Random tRNA, WT Codon usage, 100 samples

In [3]:
%%time
elongt_list = list()
ptRNA_list = list()
inputs = list()

for i in range(100):
    ptRNA_uniform = np.random.uniform(min(gr25_ptRNA),max(gr25_ptRNA),len(gr25_ptRNA))
    ptRNA_list.append(ptRNA_uniform)
    inputs.append([ptRNA_uniform,gr25_pCodon,ensmbl_latency_dict])
elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]
np.save('./data/UniformRandomtRNA_WTCodonUsage_100sample_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_100sample_ptRNA',ptRNA_list)


CPU times: user 13 s, sys: 353 ms, total: 13.3 s
Wall time: 22.2 s


## Uniform Random tRNA, WT Codon usage, 10k samples

In [4]:
%%time
elongt_list = list()
ptRNA_list = list()
inputs = list()

for i in range(10000):
    ptRNA_uniform = np.random.uniform(min(gr25_ptRNA),max(gr25_ptRNA),len(gr25_ptRNA))
    ptRNA_list.append(ptRNA_uniform)
    inputs.append([ptRNA_uniform,gr25_pCodon,ensmbl_latency_dict])
elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]
np.save('./data/UniformRandomtRNA_WTCodonUsage_10ksample_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_10ksample_ptRNA',ptRNA_list)


CPU times: user 42min 12s, sys: 21.7 s, total: 42min 34s
Wall time: 1h 5min 56s


## WT tRNA, Uniform Random Codon usage, 100 samples

In [11]:
%%time
elongt_list = list()
pCodon_list = list()
inputs = list()

syn_codon_list = [[0,1,2,3],[4,5],[6,7],[8,9,10,11],[12,13,14,15],[16,17,46,47,48,49],[18,19,42,43,44,45],
              [20,21],[22,23],[24],[25,26,27],[28,29,30,31],[32],[33],[34,35],[36,37],[38,39,54,55,56,57],[40,41],
              [50,51],[52,53],[58,59,60,61]]
        
for i in range(100):
    pCodon_uniform = np.zeros(len(gr25_pCodon))
    for syn_codons in syn_codon_list:
        pCodon_uniform[syn_codons] = np.random.uniform(min(gr25_Codon),max(gr25_Codon),len(syn_codons))
        pCodon_uniform[syn_codons] = sum(gr25_pCodon[syn_codons])/sum(pCodon_uniform[syn_codons])*pCodon_uniform[syn_codons]
    pCodon_list.append(pCodon_uniform)
    inputs.append([gr25_ptRNA, pCodon_uniform,ensmbl_latency_dict])

elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]

np.save('./data/WTtRNA_UniformRandomCodonUsage_100sample_elongt',elongt_list)
np.save('./data/WTtRNA_UniformRandomCodonUsage_100sample_pCodon',pCodon_list)


CPU times: user 12.3 s, sys: 327 ms, total: 12.6 s
Wall time: 21.9 s


## WT tRNA, Uniform Random Codon usage, 10k samples

In [6]:
%%time
elongt_list = list()
pCodon_list = list()
inputs = list()

syn_codon_list = [[0,1,2,3],[4,5],[6,7],[8,9,10,11],[12,13,14,15],[16,17,46,47,48,49],[18,19,42,43,44,45],
              [20,21],[22,23],[24],[25,26,27],[28,29,30,31],[32],[33],[34,35],[36,37],[38,39,54,55,56,57],[40,41],
              [50,51],[52,53],[58,59,60,61]]
        
for i in range(10000):
    pCodon_uniform = np.zeros(len(gr25_pCodon))
    for syn_codons in syn_codon_list:
        pCodon_uniform[syn_codons] = np.random.uniform(min(gr25_Codon),max(gr25_Codon),len(syn_codons))
        pCodon_uniform[syn_codons] = sum(gr25_pCodon[syn_codons])/sum(pCodon_uniform[syn_codons])*pCodon_uniform[syn_codons]
    pCodon_list.append(pCodon_uniform)
    inputs.append([gr25_ptRNA, pCodon_uniform,ensmbl_latency_dict])

elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]

np.save('./data/WTtRNA_UniformRandomCodonUsage_10ksample_elongt',elongt_list)
np.save('./data/WTtRNA_UniformRandomCodonUsage_10ksample_pCodon',pCodon_list)


CPU times: user 35min 9s, sys: 9.43 s, total: 35min 19s
Wall time: 48min 41s


In [7]:
## Genetic algorithm: faster tRNA distributions (Codon usage fixed, WT)

In [4]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_100sample_elongt.npy')
ptRNA_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,gr25_pCodon,elongt_list,ensmbl_latency_dict,min(gr25_ptRNA),max(gr25_ptRNA),objective='fast')
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list))
        
np.save('./data/UniformRandomtRNA_WTCodonUsage_2ksample_fasttRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_2ksample_fasttRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_2ksample_fasttRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  204.696065561943
CPU times: user 6.87 s, sys: 288 ms, total: 7.16 s
Wall time: 24.9 s


In [9]:
## Genetic algorithm: slower tRNA distributions (Codon usage fixed, WT)

In [10]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_100sample_elongt.npy')
ptRNA_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,gr25_pCodon,elongt_list,ensmbl_latency_dict,min(gr25_ptRNA),max(gr25_ptRNA),objective='slow')
    fullelongt_list.append(max(elongt_list))
    if i%100 == 0:
        print('Generation: ',i)
np.save('./data/UniformRandomtRNA_WTCodonUsage_2ksample_slowtRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_2ksample_slowtRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_2ksample_slowtRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0
Generation:  100
Generation:  200
Generation:  300
Generation:  400
Generation:  500
Generation:  600
Generation:  700
Generation:  800
Generation:  900
Generation:  1000
Generation:  1100
Generation:  1200
Generation:  1300
Generation:  1400
Generation:  1500
Generation:  1600
Generation:  1700
Generation:  1800
Generation:  1900
CPU times: user 27min 53s, sys: 3.81 s, total: 27min 57s
Wall time: 1h 33min 16s


In [ ]:
## Genetic algorithm: faster codon usage distributions (tRNA abundances fixed, WT)

In [7]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/WTtRNA_UniformRandomCodonUsage_100sample_elongt.npy')
pCodon_list = np.load('./data/WTtRNA_UniformRandomCodonUsage_100sample_pCodon.npy')
fullelongt_list = list()

for i in range(200):
    fitness, pCodon_list, elongt_list = run_ga_CodonSweep(gr25_ptRNA,pCodon_list,elongt_list,ensmbl_latency_dict,min(gr25_ptRNA),max(gr25_ptRNA),objective='fast')
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list))
        
np.save('./data/WTtRNA_UniformRandomCodonUsage_2ksample_fastCodonUsage_elongt',elongt_list)
np.save('./data/WTtRNA_UniformRandomCodonUsage_2ksample_fastCodonUsage_pCodon',pCodon_list)
np.save('./data/WTtRNA_UniformRandomCodonUsage_2ksample_fastCodonUsage_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  191.24049574247692
Generation:  100  elongt:  186.35556387395678
CPU times: user 5min 24s, sys: 3.88 s, total: 5min 28s
Wall time: 18min 53s


In [8]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/WTtRNA_UniformRandomCodonUsage_100sample_elongt.npy')
pCodon_list = np.load('./data/WTtRNA_UniformRandomCodonUsage_100sample_pCodon.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, pCodon_list, elongt_list = run_ga_CodonSweep(gr25_ptRNA,pCodon_list,elongt_list,ensmbl_latency_dict,min(gr25_ptRNA),max(gr25_ptRNA),objective='slow')
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list))
        
np.save('./data/WTtRNA_UniformRandomCodonUsage_2ksample_slowCodonUsage_elongt',elongt_list)
np.save('./data/WTtRNA_UniformRandomCodonUsage_2ksample_slowCodonUsage_pCodon',pCodon_list)
np.save('./data/WTtRNA_UniformRandomCodonUsage_2ksample_slowCodonUsage_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  193.41729437463187
Generation:  100  elongt:  208.21796451769399
Generation:  200  elongt:  208.16320069094667
Generation:  300  elongt:  207.4968291477563
Generation:  400  elongt:  206.29720074512193
CPU times: user 14min 35s, sys: 11.1 s, total: 14min 46s
Wall time: 50min 54s
